In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb


In [2]:
data = pd.read_csv("globalAirQuality.csv")
data.head()

,timestamp,country,city,latitude,longitude,pm25,pm10,no2,so2,o3,co,aqi,temperature,humidity,wind_speed
0,2025-11-04 18:25:17.554219,US,New York,40.713,-74.006,50.295,108.938,27.998,6.539,52.568,1.096,108,18.504,70.168,3.725
1,2025-11-04 19:25:17.554219,US,New York,40.713,-74.006,32.083,63.043,36.120,4.021,43.536,1.075,90,5.838,80.088,8.969
2,2025-11-04 20:25:17.554219,US,New York,40.713,-74.006,42.250,82.553,26.935,9.538,23.320,0.977,84,31.833,62.783,9.650
3,2025-11-04 21:25:17.554219,US,New York,40.713,-74.006,30.403,79.951,63.536,7.609,31.369,0.230,158,23.140,89.153,8.956
4,2025-11-04 22:25:17.554219,US,New York,40.713,-74.006,21.083,66.423,38.997,6.919,45.615,1.085,97,13.632,76.499,4.017


In [3]:
us_df = data[data['country'] == 'US'].copy()
us_df.drop(columns = ["latitude", "longitude", "country"], inplace = True)
us_df["timestamp"] = pd.to_datetime(us_df["timestamp"])

In [23]:
target = "aqi"
X = us_df.drop(columns = target)
y = us_df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
X_train_encoded = encoder.fit_transform(X_train[["city"]])

X_train_encoded = pd.DataFrame(X_train_encoded, 
                               columns=encoder.get_feature_names_out(["city"]), 
                               index = X_train.index)

X_train = pd.concat([X_train.drop(columns=["city"]), X_train_encoded], axis =1)

X_test_encoded = encoder.transform(X_test[["city"]])

X_test_encoded = pd.DataFrame(X_test_encoded, 
                              columns=encoder.get_feature_names_out(["city"]),
                              index=X_test.index)

X_test = pd.concat([X_test.drop(columns=["city"]), X_test_encoded], axis=1)

model = xgb.XGBClassifier(objective='squarederror', eval_metric='logloss', use_label_encoder=False)

model.fit(X_train, y_train)

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137], got [ 26  27  36  38  39  42  43  48  49  50  51  52  53  54  55  56  57  58
  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76
  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94
  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109 110 111 112
 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 12